https://github.com/facebookarchive/python-instagram

https://github.com/LevPasha/Instagram-API-python/blob/master/InstagramAPI/InstagramAPI.py


In [ ]:
#!pip install InstagramAPI

In [5]:
from InstagramAPI import InstagramAPI
import time
from datetime import datetime
import db_reacao

Fail to import moviepy. Need only for Video upload.


In [7]:
api = InstagramAPI("vitor.ghiotti@gmail.com", "Deb1Mis2@@")
if (api.login()):
    print("")
else:
    print("Can't login!")

Request return 429 error!
{'message': 'Please wait a few minutes before you try again.', 'status': 'fail'}
Login success!




# PESQUISA PARA ENTENDER USERID, HASHTAGS E LOCATIONS

In [9]:
query = {}
api.fbUserSearch("casariobotequim") 
query = api.LastJson

In [ ]:
print(query.keys())
print("Users".center(100,"*"))
print(query.get("users"))
print("places".center(100,"*"))
print(query.get("places"))
print("hashtags".center(100,"*"))
print(query.get("hashtags"))
print("has_more".center(100,"*"))
print(query.get("has_more"))


# INICIALIZAÇÃO DE VARIAVEIS

In [52]:
query_time_ini = datetime.strptime("2018-01-01", '%Y-%m-%d')
query_time_end = datetime.strptime("2018-04-02", '%Y-%m-%d')
query_start_collect = datetime.strptime("2018-01-01", '%Y-%m-%d')
erros_insert_comment = 0
insert_comment = 0
clientId = 1
usernameId = "1229681384"
listHashTags = ["casariobotequim"]
listLocations = ["257354853"]

# DEFINIÇÃO DE FUNÇÕES

In [92]:
def get_comments_by_usernameId(usernameId):
    more_available = True
    max_id = ''
    userFeed = {}
    
    api.getUserFeed(usernameId=usernameId,maxid=max_id)
    userFeed = api.LastJson

    while more_available:
        time.sleep(1)
        for item_feed in range(0,userFeed["num_results"]):
            if userFeed["items"][item_feed]["comment_count"]!=0:
                if userFeed["items"][item_feed]["caption"]!=None:
                    if datetime.fromtimestamp(userFeed["items"][item_feed]["caption"]["created_at"])>=query_start_collect:
                        #print(userFeed["items"][itens]["pk"])
                        #print(userFeed["items"][itens]["comment_count"])
                        #print(userFeed["items"][itens]["has_more_comments"])
                        #print(userFeed["items"][itens]["caption"]["text"])
                        #print(datetime.fromtimestamp(userFeed["items"][itens]["caption"]["created_at"]))
                        #print(userFeed["items"][itens]["caption"]["media_id"])
                        print("Inclusão de registro".center(110,"*"))
                        print("User Feed '" + usernameId + "': " +str(userFeed["items"][item_feed]["caption"]["media_id"]))
                        insert_comments(str(userFeed["items"][item_feed]["caption"]["media_id"]), "UserFeed")
                    else:
                        more_available = False
    
        if more_available:
            more_available = userFeed.get('more_available', False)
            if userFeed.get('more_available', False)==True:
                max_id = userFeed["next_max_id"]   
                api.getUserFeed(usernameId=usernameId,maxid=max_id)
                userFeed = api.LastJson
                time.sleep(2)

In [93]:
def get_comments_by_hashTags(hashTags):
    more_available = True
    max_id = ''
    hTags = {}

    for tags in range(0,len(hashTags)):
        api.getHashtagFeed(hashtagString=hashTags[tags],maxid=max_id)
        hTags = api.LastJson

        while more_available:
            time.sleep(1)
            for item_htag in range(0,len(hTags['items'])):
                if hTags['items'][item_htag]["comment_count"]!=0:
                    if hTags["items"][item_htag]["caption"]!=None:
                        if datetime.fromtimestamp(hTags["items"][item_htag]["caption"]["created_at"])>=query_start_collect:
                            print("Inclusão de registro".center(110,"*"))
                            print("HarshTag '" + hashTags[tags] + "': " +str(hTags["items"][item_htag]["caption"]["media_id"]))
                            insert_comments(str(hTags["items"][item_htag]["caption"]["media_id"]), "HashTag")
                        else:
                            more_available = False

            if more_available:   
                more_available = hTags.get('more_available', False)
                if hTags.get('more_available', False)==True:
                    max_id = hTags["next_max_id"]   
                    api.getHashtagFeed(hashtagString=hashTags[tags],maxid=max_id)
                    hTags = api.LastJson
                    time.sleep(2)

In [94]:
def get_comments_by_location(listLocations):
    
    more_available = True
    max_id = ''
    place = {}

    for index in range(0,len(listLocations)):
        api.getLocationFeed(locationId=listLocations[index],maxid=max_id)
        place = api.LastJson

        while more_available:
            time.sleep(1)
            for item_place in range(0,len(place['items'])):
                if place['items'][item_place]["comment_count"]!=0:
                    if place["items"][item_place]["caption"]!=None:
                        if datetime.fromtimestamp(place["items"][item_place]["caption"]["created_at"])>=query_start_collect:
                            print("Inclusão de registro".center(110,"*"))
                            print("Location '" + listLocations[index] + "': " +str(place["items"][item_place]["caption"]["media_id"]))
                            insert_comments(str(place["items"][item_place]["caption"]["media_id"]), "Place")
                        else:
                            more_available = False

            if more_available:   
                more_available = place.get('more_available', False)
                if place.get('more_available', False)==True:
                    max_id = place["next_max_id"]   
                    api.getLocationFeed(locationId=listLocations[index],maxid=max_id)
                    place = api.LastJson
                    time.sleep(2)

In [98]:
def insert_comments(media_id, origem):
    global erros_insert_comment
    global insert_comment
    has_more_comments = True
    max_id = ''
    comments = []
    comm={}

    while has_more_comments:
        time.sleep(1)
        api.getMediaComments(media_id, max_id=max_id)
        comm = api.LastJson

        for item_comm in range(0,len(comm['comments'])):
            if datetime.fromtimestamp(comm["comments"][item_comm]["created_at"])>=query_time_ini and datetime.fromtimestamp(comm["comments"][item_comm]["created_at"])<=query_time_end:
                if str(comm["comments"][item_comm]["user_id"])!=usernameId:
                    try:            
                        db_reacao.add_reacao_insta(clientId,4,comm["comments"][item_comm]["text"],0,comm["comments"][item_comm]["created_at"],str(comm["comments"][item_comm]["pk"]))
                        insert_comment +=1
                    except Exception as e:
                        print("Erro inclusão comentário".center(110,"*"))
                        print("Origem: " + origem)
                        print("Identificador: " + str(comm["comments"][item_comm]["pk"]))
                        print("Comentário: " + str(comm["comments"][item_comm]["text"]))
                        print("User id: " + str(usernameId))
                        print("Media ID: " + str(media_id))
                        print("Erro: " + str(e))
                        erros_insert_comment +=1

        has_more_comments = comm.get('has_more_comments', False)
        if comm.get('has_more_comments', False)==True:
            max_id = comm.get('next_max_id', '')
            time.sleep(2)

# INICIO DA COLETA

In [97]:
erros_insert_comment = 0
get_comments_by_usernameId(usernameId)
get_comments_by_hashTags(listHashTags)
get_comments_by_location(listLocations)
print("Resumo do processamento".center(110,"*"))
print("Total de inserções de comentario: " + str(insert_comment))
print("Erros de inserção de comentario: " + str(erros_insert_comment))

*********************************************Inclusão de registro*********************************************
User Feed '1229681384': 1746651296353442385
*********************************************Inclusão de registro*********************************************
User Feed '1229681384': 1742054267991890684
*********************************************Inclusão de registro*********************************************
User Feed '1229681384': 1741363201007908080
*********************************************Inclusão de registro*********************************************
User Feed '1229681384': 1737833952066129047
*********************************************Inclusão de registro*********************************************
User Feed '1229681384': 1736148281069476815
*********************************************Inclusão de registro*********************************************
User Feed '1229681384': 1732224931523748487
*********************************************Inclusão de registro*****

*******************************************Erro inclusão comentário*******************************************
Origem: HashTag
Identificador: 17915263591127165
Comentário: 🐚 🏖️ 🌺
User id: 1229681384
Media ID: 1736148281069476815
Erro: Object Reacao[new:707] cannot be stored in the database. IntegrityError: UNIQUE constraint failed: Reacao.identificador_fonte
*********************************************Inclusão de registro*********************************************
HarshTag 'casariobotequim': 1732224931523748487
*******************************************Erro inclusão comentário*******************************************
Origem: HashTag
Identificador: 17917367734128814
Comentário: 😉 😉
User id: 1229681384
Media ID: 1732224931523748487
Erro: Object Reacao[new:708] cannot be stored in the database. IntegrityError: UNIQUE constraint failed: Reacao.identificador_fonte
*********************************************Inclusão de registro*********************************************
HarshTag 'c

*******************************************Erro inclusão comentário*******************************************
Origem: HashTag
Identificador: 17907550873114271
Comentário: @paris198_ tem!!! Uhulll 💃🏼🍻🥁
User id: 1229681384
Media ID: 1701491014604319368
Erro: Object Reacao[new:731] cannot be stored in the database. IntegrityError: UNIQUE constraint failed: Reacao.identificador_fonte
*******************************************Erro inclusão comentário*******************************************
Origem: HashTag
Identificador: 17906344795120125
Comentário: Uhullllll!!!💃💃💃💃
User id: 1229681384
Media ID: 1701491014604319368
Erro: Object Reacao[new:732] cannot be stored in the database. IntegrityError: UNIQUE constraint failed: Reacao.identificador_fonte
*********************************************Inclusão de registro*********************************************
HarshTag 'casariobotequim': 1700810605893739250
*******************************************Erro inclusão comentário*******************

*******************************************Erro inclusão comentário*******************************************
Origem: Place
Identificador: 17919326713107124
Comentário: 👏👏👏
User id: 1229681384
Media ID: 1741363201007908080
Erro: Object Reacao[new:753] cannot be stored in the database. IntegrityError: UNIQUE constraint failed: Reacao.identificador_fonte
*******************************************Erro inclusão comentário*******************************************
Origem: Place
Identificador: 17932189723031219
Comentário: 😃😃
User id: 1229681384
Media ID: 1741363201007908080
Erro: Object Reacao[new:754] cannot be stored in the database. IntegrityError: UNIQUE constraint failed: Reacao.identificador_fonte
*********************************************Inclusão de registro*********************************************
Location '257354853': 1737833952066129047
*******************************************Erro inclusão comentário*******************************************
Origem: Place
Identificad

*********************************************Inclusão de registro*********************************************
Location '257354853': 1713492898856978403
*********************************************Inclusão de registro*********************************************
Location '257354853': 1713437455803242459
*********************************************Inclusão de registro*********************************************
Location '257354853': 1712753551593670266
*********************************************Inclusão de registro*********************************************
Location '257354853': 1712713206130528113
*********************************************Inclusão de registro*********************************************
Location '257354853': 1711356471185606746
*********************************************Inclusão de registro*********************************************
Location '257354853': 1710902997598691362
*********************************************Inclusão de registro*****************

*********************************************Inclusão de registro*********************************************
Location '257354853': 1691321209706004644
*********************************************Inclusão de registro*********************************************
Location '257354853': 1691301329769001671
*******************************************Erro inclusão comentário*******************************************
Origem: Place
Identificador: 17892017560153495
Comentário: Poço Semi-Artesiano? Faça sua cotação com a gente! @ofazendeirooficial !!!!
User id: 1229681384
Media ID: 1691301329769001671
Erro: Object Reacao[new:859] cannot be stored in the database. IntegrityError: UNIQUE constraint failed: Reacao.identificador_fonte
*********************************************Inclusão de registro*********************************************
Location '257354853': 1686733337802127942
*********************************************Inclusão de registro*********************************************
Lo